<a href="https://colab.research.google.com/github/Ntwenhle17/gdp-dashboard/blob/main/United_State_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas scikit-learn matplotlib seaborn aif360 fairlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, demographic_parity_difference
from aif360.datasets import AdultDataset


In [5]:
# Load Adult dataset
dataset = AdultDataset()

# Convert to pandas DataFrame
df = dataset.convert_to_dataframe()[0]

# Preview first rows
df.head()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



IOError: [Errno 2] No such file or directory: '/usr/local/lib/python3.12/dist-packages/aif360/datasets/../data/raw/adult/adult.data'
To use this class, please download the following files:

	https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
	https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
	https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

and place them, as-is, in the folder:

	/usr/local/lib/python3.12/dist-packages/aif360/data/raw/adult

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/aif360/datasets/adult_dataset.py", line 89, in __init__
    train = pd.read_csv(train_path, header=None, names=column_names,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr

TypeError: object of type 'NoneType' has no len()

In [ ]:
import pandas as pd

# Download Adult dataset directly from UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

# Column names (from dataset documentation)
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

# Load dataset
df = pd.read_csv(url, header=None, names=columns, na_values=" ?", skipinitialspace=True)

# Preview first rows
df.head()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
# One-hot encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# Train/test split with encoded features
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
# One-hot encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# Train/test split with encoded features
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))



In [ ]:
# Clean up income column (remove spaces)
df['income'] = df['income'].str.strip()

# Target label (income ≥ 50K)
y = (df['income'] == '>50K').astype(int)   # 1 = >50K, 0 = <=50K

# Features (drop target column)
X = df.drop(columns=['income'])

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

# Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, demographic_parity_difference

# Group labels for test set (Male vs Female)
groups = sensitive[X_test.index]

# Fairness metrics
metrics = {
    'selection_rate': selection_rate,
    'true_positive_rate': true_positive_rate
}

mf = MetricFrame(metrics=metrics, y_true=y_test, y_pred=y_pred, sensitive_features=groups)

print("Metrics by group:\n", mf.by_group)
print("Demographic parity difference:", demographic_parity_difference(y_test, y_pred, sensitive_features=groups))


In [ ]:
mf.by_group.plot(kind='bar', figsize=(8,6))
plt.title("Fairness metrics by sex")
plt.xticks(rotation=0)
plt.show()


In [ ]:
# Combine sex and education into one group label
groups_intersection = X.loc[X_test.index, ['sex', 'education']].astype(str).agg('-'.join, axis=1)

# Fairness metrics
metrics = {
    'selection_rate': selection_rate,
    'true_positive_rate': true_positive_rate
}

mf_intersection = MetricFrame(metrics=metrics, y_true=y_test, y_pred=y_pred, sensitive_features=groups_intersection)

print("Metrics by sex + education:\n", mf_intersection.by_group)


In [ ]:
mf_intersection.by_group.plot(kind='bar', figsize=(12,6))
plt.title("Fairness metrics by sex and education")
plt.xticks(rotation=90)
plt.show()


In [ ]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

# Mitigation: enforce demographic parity
constraint = DemographicParity()
mitigator = ExponentiatedGradient(LogisticRegression(max_iter=1000), constraints=constraint)

# Fit mitigator with sensitive attribute (sex)
mitigator.fit(X_train, y_train, sensitive_features=sensitive[X_train.index])

# Predictions after mitigation
y_pred_mitigated = mitigator.predict(X_test)

# Fairness metrics after mitigation
mf_mitigated = MetricFrame(metrics=metrics, y_true=y_test, y_pred=y_pred_mitigated, sensitive_features=groups)

print("Metrics after mitigation:\n", mf_mitigated.by_group)
print("Demographic parity difference (after):", demographic_parity_difference(y_test, y_pred_mitigated, sensitive_features=groups))


In [ ]:
# Side-by-side comparison of selection rates
comparison = pd.DataFrame({
    'Before': mf.by_group['selection_rate'],
    'After': mf_mitigated.by_group['selection_rate']
})

comparison.plot(kind='bar', figsize=(8,6))
plt.title("Selection rate before vs after mitigation")
plt.xticks(rotation=0)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Side-by-side comparison of selection rates
comparison = pd.DataFrame({
    'Before': mf.by_group['selection_rate'],
    'After': mf_mitigated.by_group['selection_rate']
})

comparison.plot(kind='bar', figsize=(8,6))
plt.title("Selection rate before vs after mitigation")
plt.xticks(rotation=0)
plt.show()


In [ ]:
# Target label (income ≥ 50K)
y = (df['income'] == ' >50K').astype(int)   # 1 = >50K, 0 = <=50K

# Features (drop target column)
X = df.drop(columns=['income'])

# Sensitive attribute (sex: Male vs Female)
sensitive = df['sex']

# Feature of interest (education level)
education = df['education']

